# From SHAP to EBM
This notebook demonstrates how to use SHAP and EBM to explain a model's predictions.
We'll use the diamonds dataset from the GGplot2 R library to train an XGBoost model to predict diamond prices.

Then, we'll use SHAP to explain the model's predictions and visualize the feature importance.
Finally, we'll train an EBM model and compare the results with the XGBoost model.

# Setup
Let's start by importing the necessary libraries and loading the data.

If you're on Colab, you should install the required libraries by running the following cell.

Moreover, you will need to upload the `diamonds.csv` file to the Colab environment and change the path to `diamonds.csv`.

In [ ]:
# !pip install --upgrade -q pandas plotly shap xgboost interpret

In [ ]:
import interpret
import pandas as pd
import plotly.express as px
import plotly.io as pio
import shap
import xgboost
from interpret.glassbox import ExplainableBoostingRegressor
from plotly.graph_objs import Figure

pio.templates.default = "plotly_white"

In [ ]:
df = (
    pd.read_csv("../data/diamonds.csv", index_col=0)
    .sample(5000, random_state=42)
    .reset_index(drop=True)
)
df.head()

# Data Processing
We first need to remove any missing values and filter out any outliers.
Moreover, we'll convert the categorical columns to ordered categorical columns.

In [ ]:
df = df[(df.x > 0) & (df.y > 0) & (df.z > 0) & (df.z < 30)]
df["cut"] = pd.Categorical(
    df["cut"],
    categories=["Fair", "Good", "Very Good", "Premium", "Ideal"],
    ordered=True,
)
df["color"] = pd.Categorical(
    df["color"], categories=["J", "I", "H", "G", "F", "E", "D"], ordered=True
)
df["clarity"] = pd.Categorical(
    df["clarity"],
    categories=["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"],
    ordered=True,
)
df.describe()

We lost 4 sample, nothing to worry about.

In [ ]:
df.info()

Now we do have categorical columns with ordered categories.

# Data Exploration
We'll create some charts to visualize the data to understand the relationships between the features.
We'll start with a scatter matrix for the numerical features and violin plots for the categorical features.
Then, we'll merge the numerical and categorical features to see how they affect the target variable by means of scatter plots.

In [ ]:
fig = px.scatter_matrix(
    df, dimensions=["carat", "depth", "table", "price", "x", "y", "z"]
)
fig.update_layout(autosize=False, width=1200, height=1200)
fig.update_traces(marker=dict(size=3, opacity=0.5))
fig.show()

In [ ]:
def plot_violin_for_variable(df: pd.DataFrame, variable: str) -> Figure:
    return px.violin(
        df,
        x=variable,
        y="price",
        color=variable,
        title="Price by Cut",
        category_orders={variable: df[variable].cat.categories.to_list()},
    )

In [ ]:
plot_violin_for_variable(df, "cut")

In [ ]:
plot_violin_for_variable(df, "color")

In [ ]:
plot_violin_for_variable(df, "clarity")

In [ ]:
px.scatter(df, x="carat", y="price", color="cut")

We can conclude that:
- The price increases with the carat size.
- The price is higher for diamonds with a better cut, color, and clarity.
- Depth and table don't seem to have a significant impact on the price.
- The dimensions x, y, and z are highly correlated with each other and the carat size.

# Modelling with XGBoost
We'll train an XGBoost model to predict the diamond prices.

Feel free to change the set of features to see what happens. In particular, you can try removing the dimensions `y` and `z`, as we saw that they are highly correlated with `x`.

Moreover, you may want to remove `depth`and `table`, as they don't seem to have a significant impact on the price.

We do not care about hyperparameter tuning in this notebook.

In [ ]:
model_df = df[
    ["carat", "cut", "color", "clarity", "depth", "table", "x", "y", "z", "price"]
].copy()

In [ ]:
train_x = model_df.drop(columns="price").sample(frac=0.8, random_state=42)
test_x = model_df.drop(columns="price").drop(train_x.index)
train_y = model_df["price"].loc[train_x.index]
test_y = model_df["price"].loc[test_x.index]
train_x.shape, test_x.shape

In [ ]:
model = xgboost.XGBRegressor(
    objective="reg:squarederror",
    max_depth=6,
    eta=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    seed=42,
    n_estimators=100,
    enable_categorical=True,
)
model.fit(train_x, train_y)
predicted_y = model.predict(test_x)
prediction_df = pd.DataFrame({"actual": test_y, "predicted": predicted_y})

In [ ]:
def plot_gof(prediction_df: pd.DataFrame):
    scatter_gof_fig = px.scatter(
        prediction_df, x="predicted", y="actual", title="Goodness of Fit"
    )
    scatter_gof_fig.add_shape(
        type="line",
        x0=0,
        y0=0,
        x1=prediction_df["predicted"].max(),
        y1=prediction_df["predicted"].max(),
    )
    scatter_gof_fig.update_layout(autosize=False, width=600, height=600)
    scatter_gof_fig.show()
    errors = prediction_df["actual"] - prediction_df["predicted"]
    px.histogram(errors, title="Error Distribution", nbins=500).update_layout(
        showlegend=False
    ).show()


def compute_metrics(prediction_df: pd.DataFrame) -> dict[str, float]:
    error = prediction_df["actual"] - prediction_df["predicted"]
    mae = error.abs().mean()
    rmse = (error**2).mean() ** 0.5
    return {"mae": mae, "rmse": rmse}


plot_gof(prediction_df)
compute_metrics(prediction_df)

The predictions are quite good, with most of the samples very close to the 45-degree line in the goodness of fit chart.

# Feature Importance
As a first test, let's see the feature importance according to the XGBoost model.

We must consider that this is an impurity-based metric, and it can suffer from several biases. In particular, it can be biased towards high cardinality features. 

In [ ]:
importance = model.get_booster().get_score(importance_type="weight")
importance_df = pd.DataFrame(
    {"Feature": list(importance.keys()), "Importance": list(importance.values())}
)

importance_df = importance_df.sort_values(by="Importance", ascending=True)

fig = px.bar(
    importance_df,
    x="Importance",
    y="Feature",
    orientation="h",
    title="Feature Importance",
)
fig.show()

# SHAP
Now, let's use SHAP to explain the model's predictions.

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer(train_x)
shap.summary_plot(shap_values, train_x, plot_type="bar", show=False)

In [ ]:
shap.summary_plot(shap_values, train_x, show=False)

In [ ]:
sample_index = 0
shap_values_test = explainer.shap_values(test_x)
shap.force_plot(
    explainer.expected_value,
    shap_values_test[sample_index, :],
    test_x.iloc[sample_index, :],
)

We see that the output is quite similar to the feature importance plot we obtained from the XGBoost model, however not identical.

Moreover, SHAP explains the direction of the effect of each feature on the prediction, which is very useful for understanding the model's behavior.

# EBM
Finally, we'll train an EBM model to predict the diamond prices and compare the results with the XGBoost model.

In [ ]:
ebm = ExplainableBoostingRegressor(random_state=42)
ebm.fit(train_x, train_y)
predicted_y_ebm = ebm.predict(test_x)
prediction_df_ebm = pd.DataFrame({"actual": test_y, "predicted": predicted_y_ebm})

In [ ]:
plot_gof(prediction_df_ebm)
compute_metrics(prediction_df_ebm)

The EBM model performs slightly worse than the XGBoost model, with a higher MAE and RMSE. This is because the EBM model is constrained to be more interpretable, and it may not capture the underlying patterns as well as the XGBoost model.

In [ ]:
ebm_global = ebm.explain_global()
interpret.show(ebm_global)

In [ ]:
ebm_local = ebm.explain_local(test_x, test_y)
interpret.show(ebm_local)

On the other hand, the EBM model is more interpretable, and we can see the effect of each feature on the prediction for each sample. Differently from SHAP, such effects are not inferred a-posteriori with some game-theory approach, but directly estimated by the model parameters.

This guarantees accurate explanations of both the global and local behavior of the model.